In [1]:
import pandas as pd
from config import DATA_DOWNLOAD_URL
import os

In [2]:
def get_market_year(row): #make a market year row in datas
    year = row['date'].year
    month = row['date'].month
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [3]:
hist = pd.read_csv(f"{DATA_DOWNLOAD_URL}/ERA5/1940-2025_vpd.csv").drop("Unnamed: 0", axis=1)

In [4]:
hist["date"] = pd.to_datetime(hist["date"])
hist = hist[hist["date"] >= "2019-09-01"] #year to start, i will take arbitrary last 5 market years
hist["MY"] = hist.apply(get_market_year, axis=1)
hist["MONTH"] = hist["date"].dt.month
hist = hist[~hist["dep"].isin([75,92,93])] #remove 3 deps cause we have no data
current = hist[hist["MY"] == "2024/2025"].copy() #copy current MY
hist = hist[hist["MY"] != "2024/2025"] #remove current MY

In [5]:
meanHist = hist[["dep", "MONTH", "vpd_min", "vpd_mean", "vpd_max"]].groupby(["dep", "MONTH"]).mean().reset_index()
meanCurrent = current[["dep", "MONTH", "vpd_min", "vpd_mean", "vpd_max"]].groupby(["dep", "MONTH"]).mean().reset_index()

In [6]:
meanCurrent

,dep,MONTH,vpd_min,vpd_mean,vpd_max
0,1,1,0.057895,0.177691,0.360193
1,1,2,0.058770,0.183886,0.404316
2,1,3,0.096471,0.354761,0.771763
3,1,4,0.230609,0.523371,0.944297
4,1,9,0.105314,0.416935,0.929223
...,...,...,...,...,...
731,95,4,0.160272,0.582396,1.084573
732,95,9,0.079830,0.369314,0.843572
733,95,10,0.037508,0.174421,0.449937
734,95,11,0.048371,0.132023,0.264044


In [7]:
meanHist.to_csv("../YieldModel/data/forecast/currentYear/vpd_mean_historical.csv")
meanCurrent.to_csv("../YieldModel/data/forecast/currentYear/vpd_mean_current.csv")